In [1]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from sklearn import svm
from datetime import datetime
import glob


#filename queue
filename_queue=glob.glob('./rawdata/data[1-4].csv')
#Create return path
return_path='./parsed/data.csv'
create_file=open(return_path,'w')

In [20]:
#Parsing data

training_set=[]
test_set=[]
training_label=[]
test_label=[]
label_count=np.zeros(16)

#return_file=open(return_path,'a')
#writer=csv.writer(return_file)
#writer.writerow()
n_data=[]

for filename in filename_queue:
    file=open(filename, newline='')
    
    reader=csv.reader(file)
    header=next(reader)
    
    for row in reader:
        row[4:]=[float(i) for i in row[4:]]
        date=datetime.strptime(row[0],'%Y/%m/%d')
        msec=row[1:3]
        labl=int(row[3])
        label_count[labl]+=1
        X_acc=list(row[4*i+4] for i in range(150))
        Y_acc=list(row[4*i+604] for i in range(150))
        Z_acc=list(row[4*i+1204] for i in range(150))
        X_gyro=list(row[4*i+1804] for i in range(150))
        Y_gyro=list(row[4*i+2404] for i in range(150))
        Z_gyro=list(row[4*i+3004] for i in range(150))
    
        window=np.array([[X_acc[0:25],X_acc[25:50],X_acc[50:75],X_acc[75:100],X_acc[100:125],X_acc[125:150]],
                [Y_acc[0:25],Y_acc[25:50],Y_acc[50:75],Y_acc[75:100],Y_acc[100:125],Y_acc[125:150]],
                [Z_acc[0:25],Z_acc[25:50],Z_acc[50:75],Z_acc[75:100],Z_acc[100:125],Z_acc[125:150]],
                [X_gyro[0:25],X_gyro[25:50],X_gyro[50:75],X_gyro[75:100],X_gyro[100:125],X_gyro[125:150]],
                [Y_gyro[0:25],Y_gyro[25:50],Y_gyro[50:75],Y_gyro[75:100],Y_gyro[100:125],Y_gyro[125:150]],
                [Z_gyro[0:25],Z_gyro[25:50],Z_gyro[50:75],Z_gyro[75:100],Z_gyro[100:125],Z_gyro[125:150]]])
    
            #5 features  what else?2
        window_mean=window.mean(axis=-1)
        window_stddev=window.std(axis=-1)
        window_median=np.median(window,axis=-1)
        window_percent25=np.percentile(window,25,axis=-1)
        window_percent75=np.percentile(window,75,axis=-1)

        window_feature=np.array([[window_mean],[window_stddev],[window_median],[window_percent25],[window_percent75]])
        window_feature=window_feature.reshape(180)
    
        if (label_count[labl]%5!=1):
            training_set.append(window_feature)
            training_label.append(labl)
        else:
            test_set.append(window_feature)
            test_label.append(labl)
        

In [21]:
#print(window_feature.shape)
#print(training_label)
print(test_set)

[array([-1.15611620e+00, -7.65894680e-01,  4.85264440e-01, -3.47102856e+00,
       -1.49752408e+00, -3.38440760e-01, -4.69950360e-01, -3.00193964e+00,
       -5.70343624e+00, -8.02390004e+00, -2.12961200e-01, -1.38755000e+00,
        9.88341020e+00,  8.53893108e+00,  5.01908892e+00,  2.38880268e+00,
        1.02983258e+01,  9.63627964e+00,  1.64360012e+00, -8.94039952e+00,
       -9.89100005e+01,  7.69440006e+01,  2.41780005e+01, -5.77920020e+00,
       -4.89999992e+00, -2.81904001e+01, -3.85840004e+00,  1.19840003e+01,
       -1.48120001e+01, -4.12439996e+00,  2.09440000e+00,  3.18108001e+01,
        7.35279984e+00, -1.23144002e+01, -1.09340001e+01, -2.41135999e+01,
        1.01999672e+00,  1.96585383e+00,  2.33341311e+00,  4.11619435e-01,
        1.89328995e+00,  1.48941451e+00,  1.04024998e+00,  3.58475766e+00,
        4.45982826e+00,  9.83955062e-01,  1.44688595e+00,  1.12749329e+00,
        7.31223183e-01,  2.32168838e+00,  5.01329024e+00,  2.20171865e+00,
        9.83567866e-01, 

In [22]:
#SVM
#prepare the data
X=training_set
y=training_label
#prepare the model
clf = sklearn.svm.SVC(gamma='scale',tol=0.1)

clf.fit(X,y)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

test_pred1=clf.predict(test_set)

ohc=OneHotEncoder(categories=[range(16)])
onehot_pred1=test_pred1.reshape(-1,1)
onehot_pred1=ohc.fit_transform(onehot_pred1).toarray()
#print(test_pred1)
#print(onehot_pred1)

print(accuracy_score(test_pred1,test_label))
#print(clf.n_support_)
#confusion_matrix(test_pred1, test_label)

0.7272727272727273


In [23]:
from sklearn.neighbors.nearest_centroid import NearestCentroid
clf = NearestCentroid()
clf.fit(X, y)
test_pred2=clf.predict(test_set)
print(accuracy_score(test_pred2,test_label))

onehot_pred2=test_pred2.reshape(-1,1)
onehot_pred2=ohc.fit_transform(onehot_pred2).toarray()
#print(onehot_pred2)
#confusion_matrix(test_pred2, test_label)

0.6212121212121212


In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
clf = RandomForestClassifier()
clf.fit(X, y)
test_pred3=clf.predict(test_set)
print(accuracy_score(test_pred3,test_label))
#print(confusion_matrix(test_pred3, test_label))
onehot_pred3=test_pred2.reshape(-1,1)
onehot_pred3=ohc.fit_transform(onehot_pred3).toarray()
#print(onehot_pred3)

0.7424242424242424


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [25]:
#ensemble 1 : unanimous consensus
ensemble1_pred= onehot_pred1*onehot_pred2*onehot_pred3
#print(ensemble1_pred)
ensemble1_pred=np.argmax(ensemble1_pred,axis=1)
print(ensemble1_pred)

[ 0  9 10  9  9  0  0 10  9  0  7  1  6  6  0 12  6  5  7 12  0  6 12  1
  0 12  0 10  0  0 12  5  0  0 12  6  0  0  5  0  1 12  6  1  5  0  0  0
 13  8  0  8  0 13  0 12  5  8  6 12  1  0  0  0  0  6]


In [26]:
#ensemble 2 : majority consensus
ensemble2_pred= (onehot_pred1+onehot_pred2+onehot_pred3)/2
ensemble2_pred= ensemble2_pred.astype('int64') 
ensemble2_pred= ensemble2_pred.astype('float64') 
ensemble2_pred=np.argmax(ensemble2_pred,axis=1)
print(ensemble2_pred)
print(test_label)
print(accuracy_score(ensemble2_pred,test_label))


[ 7  9 10  9  9  7  9 10  9  6  7  1  6  6  0 12  6  5  7 12  3  6 12  1
 10 12  2 10  9  9 12  5  0  3 12  6  9  7  5  7  1 12  6  1  5 14  3 14
 13  8 14  8 14 13  0 12  5  8  6 12  1 14  1 14  9  6]
[12, 9, 10, 10, 9, 12, 10, 10, 9, 6, 7, 1, 6, 2, 0, 0, 6, 5, 7, 3, 4, 6, 12, 1, 12, 12, 12, 0, 9, 12, 6, 5, 0, 12, 12, 6, 7, 2, 5, 0, 3, 12, 6, 1, 5, 1, 10, 13, 13, 8, 14, 8, 14, 13, 0, 12, 5, 8, 6, 12, 1, 5, 8, 3, 12, 6]
0.6212121212121212
